In [ ]:
%gui qt

import os
import warnings

import numpy as np
import progressbar
from skimage.exposure import rescale_intensity
import imageio
from distributed import LocalCluster, Client, progress

import utoolbox.utils.files as fileutils
from utoolbox.container import Volume
from utoolbox.analysis.roi import extract_mask, mask_to_contour

### Preset environment

Minimum blob size.

In [ ]:
min_size = 25000

In [ ]:
# local cluster
cluster = LocalCluster(n_workers=8, threads_per_worker=2)
client = Client(cluster)

In [ ]:
print(cluster)

### Step 2
List files.

In [ ]:
source_folder = fileutils.get_local_directory()
print(source_folder)
file_list = fileutils.list_files(
    source_folder, 
    name_filters=[fileutils.ExtensionFilter('tif')]
)

### Step 3
Generate output folder.

In [ ]:
mask_folder = os.path.join(source_folder, 'mask')
if not os.path.exists(mask_folder):
    os.makedirs(mask_folder)
    
contour_folder = os.path.join(source_folder, 'contour')
if not os.path.exists(contour_folder):
    os.makedirs(contour_folder)

### Step 4
Iterate over the volumes for the extraction.

In [ ]:
def load_and_process(file_path):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        data = Volume(file_path)
    data = rescale_intensity(data, out_range=(0., 1.))
    
    # XY MIP
    data = np.amax(data, axis=0)
    
    file_name = os.path.basename(file_path)
    
    mask = extract_mask(data, iterative=False, min_size=min_size)
    imageio.imwrite(
        os.path.join(mask_folder, file_name), 
        rescale_intensity(mask.astype(np.int8))
    )
        
    contour = mask_to_contour(mask)
    imageio.imwrite(
        os.path.join(contour_folder, file_name), 
        rescale_intensity(contour.astype(np.int8))
    )

In [ ]:
# serial
#for file_path in bar(file_list):
#    load_and_process(file_path)
#    break
    
# parallel
futures = client.map(load_and_process, file_list)